In [1]:
from active_critic.utils.gym_utils import sample_expert_transitions
import torch as th
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
import gym
from imitation.data.wrappers import RolloutInfoWrapper
from stable_baselines3.common.policies import BaseModel
from typing import Any, Dict, List, Optional, Tuple, Type, TypeVar, Union
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
import torch.nn as nn
from active_critic.utils.gym_utils import DummyExtractor
import numpy as np
from active_critic.utils.gym_utils import make_policy_dict
from metaworld.envs import ALL_V2_ENVIRONMENTS_GOAL_OBSERVABLE
from gym.wrappers import TimeLimit


/home/hendrik/miniconda3/envs/ac/lib/python3.10/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


In [ ]:
seq_len = 100
env_id = 'push'
policy_dict = make_policy_dict()
max_episode_steps = seq_len
env = ALL_V2_ENVIRONMENTS_GOAL_OBSERVABLE[policy_dict[env_id][1]]()
env._freeze_rand_vec = False
env = TimeLimit(env=env, max_episode_steps=max_episode_steps)
asd = 1

In [ ]:
class MDPLearner(gym.Env): #Wrapper?
    def __init__(self) -> None:
        super().__init__()
        #obsv space
        #action space

    def emit(self, obsv):
        pass
        #emitter forward

    def predict_emb(self, embedding, action):
        pass
        #predicter forward

    def pred_rew(self, embedding):
        pass
        #reward_predictor forward

    def learn(self, obsvs1, obsvs2, reward1, reward2, actions):
        pass
        #emit(obsv1) -> reward1*
        #emit(obsv1) -> actions -> predictions -> reward2*
        #emit(obsv2) -> reward2**
        #L1 = emit(obsv1) - emit(obsv2)
        #L2 = reward1* - reward1
        #L2 = reward2* - reward2
        #L3 = reward2 ** - reward2

    def step(self, action):
        pass
        #obsv = self.env.step(action)
        #return self.emit(obsv)

    def reset(self):
        pass
        #self.env.reset()

In [ ]:
import gym
import numpy as np

from stable_baselines3 import SAC

#env = gym.make("Hopper-v3")

model_push = SAC("MlpPolicy", env, verbose=1)
model_push.learn(total_timesteps=100*100, log_interval=4)
model_push.save("sac_push")

model = SAC.load("sac_push")

'''while True:
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)
    env.render()
    if done:
      obs = env.reset()'''

In [ ]:
env.reset()

In [ ]:
env = gym.make("Hopper-v2")


In [4]:
obs = env.reset()
rews = []
while True:
    action, _states = model_push.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)
    rews.append(reward)
    #env.render()
    if done:
      break

In [5]:
rews_np = np.array(rews)

In [7]:
rews_np

array([0.05170676, 0.05179015, 0.05188543, 0.05197164, 0.05205706,
       0.0521263 , 0.05215443, 0.05218172, 0.05215274, 0.05213872,
       0.05206964, 0.05203299, 0.05194294, 0.05189308, 0.05181692,
       0.05178132, 0.05172561, 0.05174419, 0.05194805, 0.05214596,
       0.05236517, 0.05254609, 0.05278724, 0.05293882, 0.05313413,
       0.05322903, 0.05333681, 0.05349628, 0.05359037, 0.05371782,
       0.05390673, 0.05405694, 0.05416785, 0.05436636, 0.05456583,
       0.05469297, 0.05486405, 0.0550848 , 0.05526537, 0.05539854,
       0.05562043, 0.0558463 , 0.05598457, 0.05618905, 0.0564234 ,
       0.05663836, 0.05679464, 0.05701639, 0.05726946, 0.05748555,
       0.05766134, 0.05787216, 0.0581366 , 0.05836928, 0.05858403,
       0.05879371, 0.05899074, 0.05919899, 0.05943764, 0.05969704,
       0.05991213, 0.06010548, 0.06029568, 0.06049306, 0.06071894,
       0.06097236, 0.06117186, 0.06135035, 0.06151565, 0.06171497,
       0.06193344, 0.06216329, 0.06235578, 0.06251504, 0.06266

In [6]:
rews_np[-1]

0.06685840162051389

In [ ]:
(rews_np[1:] - rews_np[:-1]).max()

In [ ]:
def make_env(env):
    def _init():
        env._freeze_rand_vec = False
        #rce = ResetCounterWrapper(env)
        riw = RolloutInfoWrapper(env)
        return riw
    return _init

In [ ]:
def make_vec_env_gym(env, num_cpu, seq_len):
    env = SubprocVecEnv([make_env(env) for i in range(num_cpu)])
    return env

In [ ]:
vec_env = make_vec_env_gym(gym.make('Hopper-v3'), 3, 3)

In [ ]:
obsv = vec_env.reset()

In [ ]:
class DummyGymPolicy(BaseModel):
    def __init__(self, observation_space: gym.spaces.Space, action_space: gym.spaces.Space, features_extractor_class: Type[BaseFeaturesExtractor] = ..., features_extractor_kwargs: Optional[Dict[str, Any]] = None, features_extractor: Optional[nn.Module] = None, normalize_images: bool = True, optimizer_class: Type[th.optim.Optimizer] = th.optim.Adam, optimizer_kwargs: Optional[Dict[str, Any]] = None):
        super().__init__(observation_space, action_space, features_extractor_class, features_extractor_kwargs, features_extractor, normalize_images, optimizer_class, optimizer_kwargs)

    def predict(
        self,
        observation: Union[th.Tensor, Dict[str, th.Tensor]],
        state: Optional[Tuple[np.ndarray, ...]] = None,
        episode_start: Optional[np.ndarray] = None,
        deterministic: bool = False,
    ) -> th.Tensor:

        result = np.array([[0,0,0]]*observation.shape[0])
        result[episode_start] = 1
        return result


In [ ]:
epsisode_start = np.array([True, True, False])
dgp = DummyGymPolicy(vec_env.observation_space, vec_env.action_space, features_extractor=DummyExtractor())

In [ ]:
actions = dgp.predict(observation=obsv, episode_start=epsisode_start)

In [ ]:
transitions = sample_expert_transitions(policy=dgp.predict, env=vec_env, episodes=10)

In [ ]:
tf = 0
for i, transition in enumerate(transitions):
    if transition['dones']:
        tf += 1

In [ ]:
tf

In [ ]:
ttf = 0
for act in transitions:
    if act['acts'][0] == 1:
        ttf+=1
ttf

In [ ]:
for k in transitions[21]['infos']['rollout']:
    print(k)

In [ ]:
obsv.shape

In [ ]:
from active_critic.model_src.transformer import *
import torch as th

In [ ]:
seq_len = 6
batch_size = 2
dim = 3

tms = ModelSetup()
tms.d_hid = 12
tms.d_model = 12
tms.d_output = 2
tms.device = 'cuda'
tms.nhead = 1
tms.nlayers = 2
tms.seq_len = seq_len

tm = TransformerModel(model_setup=tms)

In [ ]:
inpt = th.ones([batch_size, seq_len, dim], device='cuda')

In [ ]:
result = tm.forward(inpt, offset=0)

In [ ]:
result.shape